In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, Flatten, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import time

print("TensorFlow Version:", tf.__version__)
if tf.test.is_gpu_available():
    print("GPU is available!")
else:
    print("GPU not available, training on CPU.")

TensorFlow Version: 2.20.0
GPU is available!


I0000 00:00:1765708987.176882  227598 gpu_device.cc:2020] Created device /device:GPU:0 with 13551 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
DATASET_PATH = '/mnt/d/Code/DeepLearning_AOL1/data/anime_faces' #Ganti sesuai kebutuhan
IMAGE_SIZE = 64 
BATCH_SIZE = 128
NOISE_DIM = 100 

CHECKPOINT_DIR = 'mnt/d/outputs/models' #Ganti sesuai kebutuhan
os.makedirs(CHECKPOINT_DIR, exist_ok=True) 

EPOCH_CHECKPOINT_SAVE_INTERVAL = 15

In [8]:
def load_data():
    
    dataset = tf.keras.utils.image_dataset_from_directory(
        DATASET_PATH,
        labels=None, 
        image_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=True,
        interpolation='bilinear'
    )
    
    def normalize_img(image):
        image = tf.cast(image, tf.float32)
        image = (image - 127.5) / 127.5
        return image

    dataset = dataset.map(normalize_img)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return dataset

try:
    train_dataset = load_data()
except Exception as e:
    print(f"Error memuat data. Pastikan DATASET_PATH benar dan berisi gambar: {e}")

try:
    for x in train_dataset.take(1):
        print("Shape of one batch of images:", x.shape)
        break
except NameError:
    pass

Found 63565 files.


I0000 00:00:1765709047.898939  227598 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13551 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Shape of one batch of images: (128, 64, 64, 3)


In [ ]:
def build_generator(noise_dim=NOISE_DIM):
    model = Sequential(name='Generator')
    
    model.add(Dense(4 * 4 * 512, use_bias=False, input_shape=(noise_dim,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((4, 4, 512))) 
    
    model.add(Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    
    return model

generator = build_generator(NOISE_DIM)

In [ ]:
def build_discriminator(image_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)):
    model = Sequential(name='Discriminator')

    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=image_shape))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU())
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(1)) 

    return model

discriminator = build_discriminator()

In [ ]:
mse_loss = tf.keras.losses.MeanSquaredError()

REAL_TARGET = 1.0
FAKE_TARGET = 0.0
GEN_TARGET = 1.0

def discriminator_loss(real_output, fake_output):
    real_labels = tf.ones_like(real_output) * REAL_TARGET
    real_loss = mse_loss(real_labels, real_output)
    
    fake_labels = tf.ones_like(fake_output) * FAKE_TARGET
    fake_loss = mse_loss(fake_labels, fake_output)
    
    return real_loss + fake_loss

def generator_loss(fake_output):
    gen_labels = tf.ones_like(fake_output) * GEN_TARGET
    return mse_loss(gen_labels, fake_output)

generator_optimizer = Adam(1e-4) 
discriminator_optimizer = Adam(1e-4)

In [ ]:
seed = tf.random.normal([16, NOISE_DIM])

checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

manager = tf.train.CheckpointManager(checkpoint, CHECKPOINT_DIR, max_to_keep=5)

def load_latest_checkpoint():
    global initial_epoch
    initial_epoch = 0
    
    latest_checkpoint = manager.latest_checkpoint
    
    if latest_checkpoint:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Checkpoint ditemukan: {latest_checkpoint}")
        
        status = checkpoint.restore(latest_checkpoint)
        status.expect_partial() 
        
        print("Model, optimizers, dan state telah dipulihkan (secara parsial).")
        
        try:
            checkpoint_index = int(latest_checkpoint.split('-')[-1]) 
            initial_epoch = checkpoint_index * EPOCH_CHECKPOINT_SAVE_INTERVAL
            print(f"Melanjutkan training dari Epoch: {initial_epoch + 1}")
        except:
            initial_epoch = 0
            print("Tidak dapat menentukan nomor epoch sebelumnya. Melanjutkan training dari awal.")
            
    else:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] Tidak ada checkpoint ditemukan. Memulai training dari awal (Epoch 1).")
    
    return initial_epoch

initial_epoch = 0 


def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions * 0.5) + 0.5

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(tf.clip_by_value(predictions[i, :, :, :].numpy(), 0, 1))
        plt.axis('off')

    plt.savefig(f'./{CHECKPOINT_DIR}/image_at_epoch_{epoch:04d}.png')
    plt.show()

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

In [ ]:
def train(dataset, epochs_to_run):
    global initial_epoch 
    initial_epoch = load_latest_checkpoint()
    
    target_total_epochs = initial_epoch + epochs_to_run
    
    print(f"Memulai pelatihan baru. Akan berjalan selama {epochs_to_run} epoch (hingga Epoch {target_total_epochs}).")

    G_losses = []
    D_losses = []

    for epoch in range(initial_epoch, target_total_epochs): 
        start = time.time()
        current_epoch_number = epoch + 1 
        
        gen_loss_list = []
        disc_loss_list = []

        for image_batch in dataset:
            g_loss, d_loss = train_step(image_batch)
            gen_loss_list.append(g_loss.numpy())
            disc_loss_list.append(d_loss.numpy())
            
        avg_gen_loss = np.mean(gen_loss_list)
        avg_disc_loss = np.mean(disc_loss_list)
        
        G_losses.append(avg_gen_loss)
        D_losses.append(avg_disc_loss)

        generate_and_save_images(generator, current_epoch_number, seed)
        
        if current_epoch_number % EPOCH_CHECKPOINT_SAVE_INTERVAL == 0:
            save_path = manager.save()
            print(f"Checkpoint disimpan di: {save_path}")

        print (f'Epoch {current_epoch_number}, Gen Loss: {avg_gen_loss:.4f}, Disc Loss: {avg_disc_loss:.4f}, Time: {time.time()-start:.2f} sec')

    generate_and_save_images(generator, target_total_epochs, seed)
    
    return G_losses, D_losses

In [ ]:
EPOCHS_TO_RUN = 100 
history_G_loss, history_D_loss = train(train_dataset, EPOCHS_TO_RUN)

In [ ]:
start_epoch = initial_epoch + 1 
epochs_ran = len(history_G_loss)
epochs_list = list(range(start_epoch, start_epoch + epochs_ran))

plt.figure(figsize=(10, 6))
plt.plot(epochs_list, history_G_loss, label='Generator Loss', marker='o', linestyle='-')
plt.plot(epochs_list, history_D_loss, label='Discriminator Loss', marker='x', linestyle='--')

plt.title('Generator and Discriminator Loss over Epochs (LSGAN)')
plt.xlabel('Epoch')
plt.ylabel('Loss Value (Mean Squared Error)')
plt.legend()
plt.grid(True)
plt.tight_layout()